# **New York City Yellow Taxi Data**

## Objective
In this case study you will be learning exploratory data analysis (EDA) with the help of a dataset on yellow taxi rides in New York City. This will enable you to understand why EDA is an important step in the process of data science and machine learning.

## **Problem Statement**
As an analyst at an upcoming taxi operation in NYC, you are tasked to use the 2023 taxi trip data to uncover insights that could help optimise taxi operations. The goal is to analyse patterns in the data that can inform strategic decisions to improve service efficiency, maximise revenue, and enhance passenger experience.

## Tasks
You need to perform the following steps for successfully completing this assignment:
1. Data Loading
2. Data Cleaning
3. Exploratory Analysis: Bivariate and Multivariate
4. Creating Visualisations to Support the Analysis
5. Deriving Insights and Stating Conclusions

---

**NOTE:** The marks given along with headings and sub-headings are cumulative marks for those particular headings/sub-headings.<br>

The actual marks for each task are specified within the tasks themselves.

For example, marks given with heading *2* or sub-heading *2.1* are the cumulative marks, for your reference only. <br>

The marks you will receive for completing tasks are given with the tasks.

Suppose the marks for two tasks are: 3 marks for 2.1.1 and 2 marks for 3.2.2, or
* 2.1.1 [3 marks]
* 3.2.2 [2 marks]

then, you will earn 3 marks for completing task 2.1.1 and 2 marks for completing task 3.2.2.


---

## Data Understanding
The yellow taxi trip records include fields capturing pick-up and drop-off dates/times, pick-up and drop-off locations, trip distances, itemized fares, rate types, payment types, and driver-reported passenger counts.

The data is stored in Parquet format (*.parquet*). The dataset is from 2009 to 2024. However, for this assignment, we will only be using the data from 2023.

The data for each month is present in a different parquet file. You will get twelve files for each of the months in 2023.

The data was collected and provided to the NYC Taxi and Limousine Commission (TLC) by technology providers like vendors and taxi hailing apps. <br>

You can find the link to the TLC trip records page here: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

###  Data Description
You can find the data description here: [Data Dictionary](https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf)

**Trip Records**



|Field Name       |description |
|:----------------|:-----------|
| VendorID | A code indicating the TPEP provider that provided the record. <br> 1= Creative Mobile Technologies, LLC; <br> 2= VeriFone Inc. |
| tpep_pickup_datetime | The date and time when the meter was engaged.  |
| tpep_dropoff_datetime | The date and time when the meter was disengaged.   |
| Passenger_count | The number of passengers in the vehicle. <br> This is a driver-entered value. |
| Trip_distance | The elapsed trip distance in miles reported by the taximeter. |
| PULocationID | TLC Taxi Zone in which the taximeter was engaged |
| DOLocationID | TLC Taxi Zone in which the taximeter was disengaged |
|RateCodeID |The final rate code in effect at the end of the trip.<br> 1 = Standard rate <br> 2 = JFK <br> 3 = Newark <br>4 = Nassau or Westchester <br>5 = Negotiated fare <br>6 = Group ride |
|Store_and_fwd_flag |This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, aka “store and forward,” because the vehicle did not have a connection to the server.  <br>Y= store and forward trip <br>N= not a store and forward trip |
|Payment_type| A numeric code signifying how the passenger paid for the trip. <br> 1 = Credit card <br>2 = Cash <br>3 = No charge <br>4 = Dispute <br>5 = Unknown <br>6 = Voided trip |
|Fare_amount| The time-and-distance fare calculated by the meter. <br>Extra Miscellaneous extras and surcharges.  Currently, this only includes the 0.50 and 1 USD rush hour and overnight charges. |
|MTA_tax |0.50 USD MTA tax that is automatically triggered based on the metered rate in use. |
|Improvement_surcharge | 0.30 USD improvement surcharge assessed trips at the flag drop. The improvement surcharge began being levied in 2015. |
|Tip_amount |Tip amount – This field is automatically populated for credit card tips. Cash tips are not included. |
| Tolls_amount | Total amount of all tolls paid in trip.  |
| total_amount | The total amount charged to passengers. Does not include cash tips. |
|Congestion_Surcharge |Total amount collected in trip for NYS congestion surcharge. |
| Airport_fee | 1.25 USD for pick up only at LaGuardia and John F. Kennedy Airports|

Although the amounts of extra charges and taxes applied are specified in the data dictionary, you will see that some cases have different values of these charges in the actual data.

**Taxi Zones**

Each of the trip records contains a field corresponding to the location of the pickup or drop-off of the trip, populated by numbers ranging from 1-263.

These numbers correspond to taxi zones, which may be downloaded as a table or map/shapefile and matched to the trip records using a join.

This is covered in more detail in later sections.

---

## **1** Data Preparation

<font color = red>[5 marks]</font> <br>

### Import Libraries

In [1]:
# Import warnings
import warnings
warnings.filterwarnings('ignore')


In [2]:
# Import the libraries you will be using for analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Recommended versions
# numpy version: 1.26.4
# pandas version: 2.2.2
# matplotlib version: 3.10.0
# seaborn version: 0.13.2

# Check versions
print("numpy version:", np.__version__)
print("pandas version:", pd.__version__)
print("matplotlib version:", plt.matplotlib.__version__)
print("seaborn version:", sns.__version__)

numpy version: 1.26.4
pandas version: 2.2.2
matplotlib version: 3.9.2
seaborn version: 0.13.2


### **1.1** Load the dataset
<font color = red>[5 marks]</font> <br>

You will see twelve files, one for each month.

To read parquet files with Pandas, you have to follow a similar syntax as that for CSV files.

`df = pd.read_parquet('file.parquet')`

In [4]:
# Try loading one file

# df = pd.read_parquet('2023-1.parquet')
# df.info()
df = pd.read_parquet(r"C:\Users\Yatin\OneDrive\Desktop\Datasets and Dictionary-NYC\Datasets and Dictionary\trip_records\2023-1.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3041714 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee            floa

How many rows are there? Do you think handling such a large number of rows is computationally feasible when we have to combine the data for all twelve months into one?

To handle this, we need to sample a fraction of data from each of the files. How to go about that? Think of a way to select only some portion of the data from each month's file that accurately represents the trends.

In [5]:
#How many rows are there?

num_rows = df.shape[0]
print(f"Number of rows in 2023-1 data: {num_rows}")

Number of rows in 2023-1 data: 3041714


#### Sampling the Data
> One way is to take a small percentage of entries for pickup in every hour of a date. So, for all the days in a month, we can iterate through the hours and select 5% values randomly from those. Use `tpep_pickup_datetime` for this. Separate date and hour from the datetime values and then for each date, select some fraction of trips for each of the 24 hours.

To sample data, you can use the `sample()` method. Follow this syntax:

```Python
# sampled_data is an empty DF to keep appending sampled data of each hour
# hour_data is the DF of entries for an hour 'X' on a date 'Y'

sample = hour_data.sample(frac = 0.05, random_state = 42)
# sample 0.05 of the hour_data
# random_state is just a seed for sampling, you can define it yourself

sampled_data = pd.concat([sampled_data, sample]) # adding data for this hour to the DF
```

This *sampled_data* will contain 5% values selected at random from each hour.

Note that the code given above is only the part that will be used for sampling and not the complete code required for sampling and combining the data files.

Keep in mind that you sample by date AND hour, not just hour. (Why?)

---

**1.1.1** <font color = red>[5 marks]</font> <br>
Figure out how to sample and combine the files.

**Note:** It is not mandatory to use the method specified above. While sampling, you only need to make sure that your sampled data represents the overall data of all the months accurately.

In [6]:
# Sample the data
# It is recommmended to not load all the files at once to avoid memory overload

In [7]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Take a small percentage of entries from each hour of every date.
# Iterating through the monthly data:
#   read a month file -> day -> hour: append sampled data -> move to next hour -> move to next day after 24 hours -> move to next month file
# Create a single dataframe for the year combining all the monthly data

# Select the folder having data files
import os

# Select the folder having data files
folder_path = r"C:\Users\Yatin\OneDrive\Desktop\Datasets and Dictionary-NYC\Datasets and Dictionary\trip_records"
os.chdir(folder_path)

# Create a list of all the twelve files to read
file_list = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f.lower().endswith('.parquet')]

# initialise an empty dataframe
sampled_df = pd.DataFrame()


# iterate through the list of files and sample one by one:
for file_name in file_list:
    try:
        # file path for the current file
        file_path = os.path.join(os.getcwd(), file_name)

        # Reading the current file
        df = pd.read_parquet(file_path)

        df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
        df['date'] =df['tpep_pickup_datetime'].dt.date
        df['hour'] =df['tpep_pickup_datetime'].dt.hour

        
        # We will store the sampled data for the current date in this df by appending the sampled data from each hour to this
        # After completing iteration through each date, we will append this data to the final dataframe.
        sampled_data = pd.DataFrame()

        # Loop through dates and then loop through every hour of each date
        for date in df['date'].unique():
            date_data = df[df['date'] == date]
            
            # Iterate through each hour of the selected date
            for hour in range(24):
                hour_data = date_data[date_data['hour'] == hour]
                
                # Sample 5% of the hourly data randomly
                sample = pd.DataFrame()
                
                if not hour_data.empty:
                    sample = hour_data.sample(frac=0.0075, random_state=42)

                # add data of this hour to the dataframe
                    sampled_data = pd.concat([sampled_data,sample])
                
        # Concatenate the sampled data of all the dates to a single dataframe
                sampled_df = pd.concat([sampled_df, sampled_data])


    except Exception as e:
        print(f"Error reading file {file_name}: {e}")

After combining the data files into one DataFrame, convert the new DataFrame to a CSV or parquet file and store it to use directly.

Ideally, you can try keeping the total entries to around 250,000 to 300,000.

In [ ]:
# Store the df in csv/parquet
# df.to_parquet('')
sampled_df.to_parquet("sampled_df.parquet", index=False)

## **2** Data Cleaning
<font color = red>[30 marks]</font> <br>

Now we can load the new data directly.

In [ ]:
# Load the new data file
df = pd.read_parquet("sampled_df.parquet")

In [ ]:
# df.head()
df.head()

In [ ]:
df.info()

#### **2.1** Fixing Columns
<font color = red>[10 marks]</font> <br>

Fix/drop any columns as you seem necessary in the below sections

**2.1.1** <font color = red>[2 marks]</font> <br>

Fix the index and drop unnecessary columns

In [ ]:
# Fix the index and drop any columns that are not needed
df.reset_index(drop=True, inplace=True)
columns_to_drop = ['store_and_fwd_flag', 'mta_tax', 'tolls_amount', 'extra']
df.drop(columns=columns_to_drop, inplace=True)
df.head()

**2.1.2** <font color = red>[3 marks]</font> <br>
There are two airport fee columns. This is possibly an error in naming columns. Let's see whether these can be combined into a single column.

In [ ]:
# Combine the two airport fee columns
diffs = df[df['Airport_fee'] != df['airport_fee']]
print(f"Rows with different values: {len(diffs)}")
print("Null counts:")
print(df[['Airport_fee', 'airport_fee']].isnull().sum())
df['airport_fee_combined'] = df['Airport_fee'].combine_first(df['airport_fee'])
df.drop(columns=['Airport_fee', 'airport_fee'], inplace=True)
df.rename(columns={'airport_fee_combined': 'airport_fee'}, inplace=True)

**2.1.3** <font color = red>[5 marks]</font> <br>
Fix columns with negative (monetary) values

In [ ]:
# check where values of fare amount are negative
negative_fares = df[df['fare_amount'] < 0]
print(f"Number of negative fare amounts: {len(negative_fares)}")
print(negative_fares.head())

Did you notice something different in the `RatecodeID` column for above records?

In [ ]:
# Analyse RatecodeID for the negative fare amounts
negative_fares = df[df['fare_amount'] < 0]
ratecode_analysis = negative_fares['RatecodeID'].value_counts()
print(ratecode_analysis)

In [ ]:
# Find which columns have negative values
numeric_df = df.select_dtypes(include=['float64', 'int64'])
negative_values = numeric_df[numeric_df < 0].count()
columns_with_negative_values = negative_values[negative_values > 0]

print("Columns with negative values:")
print(columns_with_negative_values)

In [ ]:
# fix these negative values
negative_cols = ['fare_amount', 'tip_amount', 'improvement_surcharge',
                 'total_amount', 'congestion_surcharge', 'airport_fee']
for col in negative_cols:
    df.loc[df[col] < 0, col] = df[col].abs()
df.head()



### **2.2** Handling Missing Values
<font color = red>[10 marks]</font> <br>

**2.2.1**  <font color = red>[2 marks]</font> <br>
Find the proportion of missing values in each column




In [ ]:
# Find the proportion of missing values in each column
def render_bar_plot(x, y, title, xlabel, ylabel, figsize=(12, 6), show_data_labels=False):
    plt.figure(figsize=figsize)
    ax = sns.barplot(x=x, y=y)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xticks(rotation=90)

    if show_data_labels:
        for p in ax.patches:
            ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', xytext=(0, 9), textcoords='offset points')

    plt.show()

missing_values = df.isna().mean() * 100
render_bar_plot(missing_values.index, missing_values.values, 'Proportion of Missing Values in Each Column', 'Columns', 'Proportion of Missing Values (%)', show_data_labels=True)



**2.2.2**  <font color = red>[3 marks]</font> <br>
Handling missing values in `passenger_count`

In [ ]:
# Display the rows with null values
# Impute NaN values in 'passenger_count'
rows_with_nulls = df[df.isnull().any(axis=1)]
print("Rows with missing values:")
print(rows_with_nulls)

# Impute NaN values in 'passenger_count'
df['passenger_count'] = df['passenger_count'].fillna(df['passenger_count'].median())
print(f"Missing values in 'passenger_count' after imputation: {df['passenger_count'].isnull().sum()}")


Did you find zeroes in passenger_count? Handle these.

**2.2.3**  <font color = red>[2 marks]</font> <br>
Handle missing values in `RatecodeID`

In [ ]:
# Fix missing values in 'RatecodeID'
mode_ratecode = df['RatecodeID'].mode()[0]
df['RatecodeID'].fillna(mode_ratecode, inplace=True)
missing_ratecode = df['RatecodeID'].isnull().sum()
print(f"Remaining missing values in 'RatecodeID': {missing_ratecode}")


**2.2.4**  <font color = red>[3 marks]</font> <br>
Impute NaN in `congestion_surcharge`

In [ ]:
# handle null values in congestion_surcharge
df['congestion_surcharge'] = df['congestion_surcharge'].fillna(0)
print(f"Remaining missing values in 'congestion_surcharge': {df['congestion_surcharge'].isnull().sum()}")

Are there missing values in other columns? Did you find NaN values in some other set of columns? Handle those missing values below.

In [ ]:
# Handle any remaining missing values
# Handle missing values in 'RatecodeID' by filling with the most frequent value (mode)
mode_ratecode = df['RatecodeID'].mode()[0]
df['RatecodeID'].fillna(mode_ratecode, inplace=True)
df['congestion_surcharge'].fillna(0, inplace=True)
df['airport_fee'].fillna(0, inplace=True)
missing_values = df.isnull().sum()
print("Missing values after imputation:")
print(missing_values[missing_values > 0])  # Should show no missing values


### **2.3** Handling Outliers
<font color = red>[10 marks]</font> <br>

Before we start fixing outliers, let's perform outlier analysis.

In [ ]:
# Describe the data and check if there are any potential outliers present
# Check for potential out of place values in various columns
# Describe the data and check if there are any potential outliers present
# Check for potential out of place values in various columns

# Descriptive statistics for numeric columns
desc_stats = df.describe()

# Describe the data and check for potential outliers
import seaborn as sns
import matplotlib.pyplot as plt

# Descriptive statistics for numeric columns
desc_stats = df.describe()
print("Descriptive Statistics:")
print(desc_stats)

# Define potential numeric columns to check
possible_numeric_columns = ['fare_amount', 'tip_amount', 'tolls_amount',
                            'trip_distance', 'total_amount', 'congestion_surcharge']

# Filter only columns that actually exist in df
numeric_columns = [col for col in possible_numeric_columns if col in df.columns]

# Boxplots for all existing numeric columns
plt.figure(figsize=(15, 10))
for i, col in enumerate(numeric_columns, 1):
    plt.subplot(2, 3, i)
    sns.boxplot(data=df, x=col)
    plt.title(f'Boxplot of {col}')
plt.tight_layout()
plt.show()

# Histograms for all existing numeric columns
plt.figure(figsize=(15, 10))
for i, col in enumerate(numeric_columns, 1):
    plt.subplot(2, 3, i)
    sns.histplot(df[col], kde=True)
    plt.title(f'Histogram of {col}')
plt.tight_layout()
plt.show()

# Check for out-of-place values in known columns
# Only check for columns that exist
conditions = []
if 'trip_distance' in df.columns:
    conditions.append(df['trip_distance'] < 0)
if 'fare_amount' in df.columns:
    conditions.append(df['fare_amount'] < 0)
if 'tip_amount' in df.columns:
    conditions.append(df['tip_amount'] < 0)
if 'tolls_amount' in df.columns:
    conditions.append(df['tolls_amount'] < 0)

# Combine conditions for negative value check
if conditions:
    negative_values = df[conditions[0]]
    for cond in conditions[1:]:
        negative_values = pd.concat([negative_values, df[cond]])
else:
    negative_values = pd.DataFrame()

# Check for invalid payment types if column exists
if 'payment_type' in df.columns:
    invalid_payment_types = df[~df['payment_type'].isin([1, 2, 3, 4])]
else:
    invalid_payment_types = pd.DataFrame()

# Display results
print("\nRows with negative values in restricted columns:")
print(negative_values)

print("\nRows with invalid payment types:")
print(invalid_payment_types)


**2.3.1**  <font color = red>[10 marks]</font> <br>
Based on the above analysis, it seems that some of the outliers are present due to errors in registering the trips. Fix the outliers.

Some points you can look for:
- Entries where `trip_distance` is nearly 0 and `fare_amount` is more than 300
- Entries where `trip_distance` and `fare_amount` are 0 but the pickup and dropoff zones are different (both distance and fare should not be zero for different zones)
- Entries where `trip_distance` is more than 250  miles.
- Entries where `payment_type` is 0 (there is no payment_type 0 defined in the data dictionary)

These are just some suggestions. You can handle outliers in any way you wish, using the insights from above outlier analysis.

How will you fix each of these values? Which ones will you drop and which ones will you replace?

First, let us remove 7+ passenger counts as there are very less instances.

In [ ]:
# remove passenger_count > 6
df = df[df['passenger_count'] <= 6]

# Remove rows where passenger_count is 0
df = df[df['passenger_count'] >= 0]

sns.histplot(y=df['passenger_count'], data=df)
plt.show()

In [ ]:
# Continue with outlier handling
df = df[df['payment_type'] != 0]
sns.boxplot(y=df['payment_type'], data=df)
plt.show()

In [ ]:
outlier_years = df[(df['tpep_pickup_datetime'].dt.year != 2023) | (df['tpep_dropoff_datetime'].dt.year != 2023)]
outlier_years

In [ ]:
plt.figure(figsize=(10, 3))

sns.boxplot(data=df, x='trip_distance')
plt.xlabel('Distance (mile)')
plt.title('Trip Distance Distribution')

plt.show()

In [ ]:
#***Assuming that taxis that travel less than (0.62 miles) are incorrect records such as the possibility of passengers or drivers canceling orders, then datasets that have this value will be dropped.
print('The amount of data that has a distance of less than 1 km (0.62 miles):', len(df[df['trip_distance'] < 0.62]))

In [ ]:
# Remove rows where trip_distance is less than 0.62 miles
df = df[df['trip_distance'] > 0.62]

In [ ]:
print('The amount of data that has a distance of more than (120 miles):', len(df[df['trip_distance'] > 120]))

In [ ]:
# Remove rows where trip_distance is more than 120 miles
df = df[df['trip_distance'] < 120]

In [ ]:
# Do any columns need standardising?


## **3** Exploratory Data Analysis
<font color = red>[90 marks]</font> <br>

In [ ]:
df.columns.tolist()

#### **3.1** General EDA: Finding Patterns and Trends
<font color = red>[40 marks]</font> <br>

**3.1.1** <font color = red>[3 marks]</font> <br>
Categorise the varaibles into Numerical or Categorical.
* `VendorID`:
* `tpep_pickup_datetime`:
* `tpep_dropoff_datetime`:
* `passenger_count`:
* `trip_distance`:
* `RatecodeID`:
* `PULocationID`:
* `DOLocationID`:
* `payment_type`:
* `pickup_hour`:
* `trip_duration`:


The following monetary parameters belong in the same category, is it categorical or numerical?


* `fare_amount`
* `extra`
* `mta_tax`
* `tip_amount`
* `tolls_amount`
* `improvement_surcharge`
* `total_amount`
* `congestion_surcharge`
* `airport_fee`

##### Temporal Analysis

**3.1.2** <font color = red>[5 marks]</font> <br>
Analyse the distribution of taxi pickups by hours, days of the week, and months.

In [ ]:
# Find and show the hourly trends in taxi pickups
df['pickup_time_derived'] = df['tpep_pickup_datetime'].dt.hour
hourly_pickups = df['pickup_time_derived'].value_counts().sort_index()
render_bar_plot(hourly_pickups.index, hourly_pickups.values, 'Hourly Trends in Taxi Pickups', 'Hour of the Day', 'Number of Pickups')


In [ ]:
# Find and show the daily trends in taxi pickups (days of the week)
df['pickup_day_derived'] = df['tpep_pickup_datetime'].dt.dayofweek
day_name_map = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
df['pickup_day_name_derived'] = df['pickup_day_derived'].map(day_name_map)
daily_pickups = df['pickup_day_name_derived'].value_counts().reindex(day_name_map.values())
render_bar_plot(daily_pickups.index, daily_pickups.values, 'Daily Trends in Taxi Pickups', 'Day of the Week', 'Number of Pickups')



In [ ]:
# Show the monthly trends in pickups
# Show the monthly trends in pickups

def render_line_plot(x, y, title, xlabel, ylabel, figsize=(12, 6)):
    plt.figure(figsize=figsize)
    sns.lineplot(x=x, y=y, marker='o')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.xticks(rotation=45)
    plt.show()
df['pickup_month_derived'] = df['tpep_pickup_datetime'].dt.month
month_name_map = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}
df['pickup_month_name_derived'] = df['pickup_month_derived'].map(month_name_map)
monthly_pickups = df['pickup_month_name_derived'].value_counts().reindex(month_name_map.values())
render_line_plot(monthly_pickups.index.astype(str), monthly_pickups.values, 'Monthly Trends in Taxi Pickups', 'Month', 'Number of Pickups')

##### Financial Analysis

Take a look at the financial parameters like `fare_amount`, `tip_amount`, `total_amount`, and also `trip_distance`. Do these contain zero/negative values?

In [ ]:
# Analyse the above parameters
financial_columns = ['fare_amount', 'tip_amount', 'total_amount', 'trip_distance']
df[financial_columns].describe()

Do you think it is beneficial to create a copy DataFrame leaving out the zero values from these?

**3.1.3** <font color = red>[2 marks]</font> <br>
Filter out the zero values from the above columns.

**Note:** The distance might be 0 in cases where pickup and drop is in the same zone. Do you think it is suitable to drop such cases of zero distance?

In [ ]:
# Create a df with non zero entries for the selected parameters.
non_zero_df = df[(df['fare_amount'] != 0) & (df['tip_amount'] != 0) & (df['total_amount'] != 0) & (df['trip_distance'] != 0)]
non_zero_df.head()

**3.1.4** <font color = red>[3 marks]</font> <br>
Analyse the monthly revenue (`total_amount`) trend

In [ ]:
# Group data by month and analyse monthly revenue
monthly_revenue = df.groupby('pickup_month_name_derived')['total_amount'].sum().reindex(month_name_map.values())
render_line_plot(monthly_revenue.index, monthly_revenue.values, 'Monthly Revenue Trend', 'Month', 'Total Revenue')

**3.1.5** <font color = red>[3 marks]</font> <br>
Show the proportion of each quarter of the year in the revenue

In [ ]:
# Calculate proportion of each quarter
def month_to_quarter(month):
    if month in [1, 2, 3]:
        return 'Q1'
    elif month in [4, 5, 6]:
        return 'Q2'
    elif month in [7, 8, 9]:
        return 'Q3'
    else:
        return 'Q4'
non_zero_df['pickup_quarter_derived'] = non_zero_df['pickup_month_derived'].apply(month_to_quarter)
quarterly_revenue = non_zero_df.groupby('pickup_quarter_derived')['total_amount'].sum()
plt.figure(figsize=(10, 8))
plt.pie(quarterly_revenue, labels=quarterly_revenue.index, autopct='%1.1f%%', startangle=140)
plt.title('Quarterly Revenue Distribution')
plt.axis('equal')
plt.show()


**3.1.6** <font color = red>[3 marks]</font> <br>
Visualise the relationship between `trip_distance` and `fare_amount`. Also find the correlation value for these two.

**Hint:** You can leave out the trips with trip_distance = 0

In [ ]:
# Show how trip fare is affected by distance
def render_scatter_plot(x, y, data, title, xlabel, ylabel, figsize=(12, 6)):
    plt.figure(figsize=figsize)
    sns.scatterplot(x=x, y=y, data=data, alpha=0.5)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()
filtered_df = df[df['trip_distance'] > 0]
render_scatter_plot('trip_distance', 'fare_amount', filtered_df, 'Relationship between Trip Distance and Fare Amount', 'Trip Distance (miles)', 'Fare Amount')
correlation_value = filtered_df['trip_distance'].corr(filtered_df['fare_amount'])
print(f'Correlation between Trip Distance and Fare Amount: {correlation_value:.2f}')

**3.1.7** <font color = red>[5 marks]</font> <br>
Find and visualise the correlation between:
1. `fare_amount` and trip duration (pickup time to dropoff time)
2. `fare_amount` and `passenger_count`
3. `tip_amount` and `trip_distance`

In [ ]:
# Show relationship between fare and trip duration
df['trip_duration_derived'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60
render_scatter_plot('trip_duration_derived', 'fare_amount', df, 'Relationship between Trip Duration and Fare Amount', 'Trip Duration (minutes)', 'Fare Amount')
correlation_value_duration_fare = df['trip_duration_derived'].corr(df['fare_amount'])
print(f'Correlation between Trip Duration and Fare Amount: {correlation_value_duration_fare:.2f}')

In [ ]:
# Show relationship between fare and number of passengers
render_scatter_plot('passenger_count', 'fare_amount', df, 'Relationship between Number of Passengers and Fare Amount', 'Number of Passengers', 'Fare Amount')
correlation_value_passenger_fare = df['passenger_count'].corr(df['fare_amount'])
print(f'Correlation between Number of Passengers and Fare Amount: {correlation_value_passenger_fare:.2f}')

In [ ]:
# Show relationship between tip and trip distance
render_scatter_plot('trip_distance', 'tip_amount', df, 'Relationship between Trip Distance and Tip Amount', 'Trip Distance (miles)', 'Tip Amount')
correlation_value_tip_distance = df['trip_distance'].corr(df['tip_amount'])
print(f'Correlation between Trip Distance and Tip Amount: {correlation_value_tip_distance:.2f}')

**3.1.8** <font color = red>[3 marks]</font> <br>
Analyse the distribution of different payment types (`payment_type`)

In [ ]:
# Analyse the distribution of different payment types (payment_type).
payment_type_counts = non_zero_df['payment_type'].value_counts()
payment_type_map = {
    1: 'Credit Card',
    2: 'Cash',
    3: 'No Charge',
    4: 'Dispute',
    5: 'Unknown',
    6: 'Voided Trip'
}
payment_type_counts.index = payment_type_counts.index.map(payment_type_map)
render_bar_plot(payment_type_counts.index, payment_type_counts.values, 'Distribution of Payment Types', 'Payment Type', 'Count', show_data_labels=True)

- 1= Credit card
- 2= Cash
- 3= No charge
- 4= Dispute



##### Geographical Analysis

For this, you have to use the *taxi_zones.shp* file from the *taxi_zones* folder.

There would be multiple files inside the folder (such as *.shx, .sbx, .sbn* etc). You do not need to import/read any of the files other than the shapefile, *taxi_zones.shp*.

Do not change any folder structure - all the files need to be present inside the folder for it to work.

The folder structure should look like this:
```
Taxi Zones
|- taxi_zones.shp.xml
|- taxi_zones.prj
|- taxi_zones.sbn
|- taxi_zones.shp
|- taxi_zones.dbf
|- taxi_zones.shx
|- taxi_zones.sbx

 ```

 You only need to read the `taxi_zones.shp` file. The *shp* file will utilise the other files by itself.

We will use the *GeoPandas* library for geopgraphical analysis
```
import geopandas as gpd
```

More about geopandas and shapefiles: [About](https://geopandas.org/en/stable/about.html)


Reading the shapefile is very similar to *Pandas*. Use `gpd.read_file()` function to load the data (*taxi_zones.shp*) as a GeoDataFrame. Documentation: [Reading and Writing Files](https://geopandas.org/en/stable/docs/user_guide/io.html)

In [ ]:
# !pip install geopandas
!pip install geopandas

**3.1.9** <font color = red>[2 marks]</font> <br>
Load the shapefile and display it.

In [ ]:
# import geopandas as gpd
import geopandas as gpd

# Read the shapefile using geopandas
file_path = r'/taxi_zones.shp'
zones = gpd.read_file(r'/taxi_zones.shp')
zones.head()

Now, if you look at the DataFrame created, you will see columns like: `OBJECTID`,`Shape_Leng`, `Shape_Area`, `zone`, `LocationID`, `borough`, `geometry`.
<br><br>

Now, the `locationID` here is also what we are using to mark pickup and drop zones in the trip records.

The geometric parameters like shape length, shape area and geometry are used to plot the zones on a map.

This can be easily done using the `plot()` method.

In [ ]:
# print(zones.info())
# zones.plot()
zones.info()
zones.plot(figsize=(12, 12))

Now, you have to merge the trip records and zones data using the location IDs.



**3.1.10** <font color = red>[3 marks]</font> <br>
Merge the zones data into trip data using the `locationID` and `PULocationID` columns.

In [ ]:
# Merge zones and trip records using locationID and PULocationID
merged_df = non_zero_df.merge(zones, left_on='PULocationID', right_on='LocationID', how='left')

# Display the first few rows of the merged DataFrame
merged_df.head()

**3.1.11** <font color = red>[3 marks]</font> <br>
Group data by location IDs to find the total number of trips per location ID

In [ ]:
# Group data by location and calculate the number of trips
trips_per_location = merged_df.groupby('PULocationID').size().reset_index(name='total_trips')
trips_per_location.head()

**3.1.12** <font color = red>[2 marks]</font> <br>
Now, use the grouped data to add number of trips to the GeoDataFrame.

We will use this to plot a map of zones showing total trips per zone.

In [ ]:
# Merge trip counts back to the zones GeoDataFrame
zones_with_trips = zones.merge(trips_per_location, left_on='LocationID', right_on='PULocationID', how='left')
zones_with_trips['total_trips'].fillna(0, inplace=True)
zones_with_trips.plot(column='total_trips', cmap='OrRd', legend=True, legend_kwds={'label': "Total Trips per Zone", 'orientation': "horizontal"}, figsize=(12, 12))
plt.title('Total Trips per Zone')
plt.show()

The next step is creating a color map (choropleth map) showing zones by the number of trips taken.

Again, you can use the `zones.plot()` method for this. [Plot Method GPD](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.plot.html#geopandas.GeoDataFrame.plot)

But first, you need to define the figure and axis for the plot.

`fig, ax = plt.subplots(1, 1, figsize = (12, 10))`

This function creates a figure (fig) and a single subplot (ax)

---

After setting up the figure and axis, we can proceed to plot the GeoDataFrame on this axis. This is done in the next step where we use the plot method of the GeoDataFrame.

You can define the following parameters in the `zones.plot()` method:
```
column = '',
ax = ax,
legend = True,
legend_kwds = {'label': "label", 'orientation': "<horizontal/vertical>"}
```

To display the plot, use `plt.show()`.

**3.1.13** <font color = red>[3 marks]</font> <br>
Plot a color-coded map showing zone-wise trips

In [ ]:
# Define figure and axis
zone_trip_counts = merged_df['PULocationID'].value_counts().reset_index()
zone_trip_counts.columns = ['LocationID', 'trip_count']
zones = zones.merge(zone_trip_counts, on='LocationID', how='left')
zones['trip_count'] = zones['trip_count'].fillna(0)
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

# Plot the map and display it
zones.plot(column='trip_count', ax=ax, legend=True, legend_kwds={'label': "Number of Trips", 'orientation': "horizontal"}, cmap='OrRd')
plt.title('Zone-wise Number of Trips')
plt.show()

In [ ]:
# can you try displaying the zones DF sorted by the number of trips?
zones_sorted = zones.sort_values(by='trip_count', ascending=False)
zones_sorted[['zone', 'trip_count']]

Here we have completed the temporal, financial and geographical analysis on the trip records.

**Compile your findings from general analysis below:**

You can consider the following points:

* Busiest hours, days and months
* Trends in revenue collected
* Trends in quarterly revenue
* How fare depends on trip distance, trip duration and passenger counts
* How tip amount depends on trip distance
* Busiest zones


#### **3.2** Detailed EDA: Insights and Strategies
<font color = red>[50 marks]</font> <br>

Having performed basic analyses for finding trends and patterns, we will now move on to some detailed analysis focussed on operational efficiency, pricing strategies, and customer experience.

##### Operational Efficiency

Analyze variations by time of day and location to identify bottlenecks or inefficiencies in routes

**3.2.1** <font color = red>[3 marks]</font> <br>
Identify slow routes by calculating the average time taken by cabs to get from one zone to another at different hours of the day.

Speed on a route *X* for hour *Y* = (*distance of the route X / average trip duration for hour Y*)

In [ ]:
# Find routes which have the slowest speeds at different times of the day
import pandas as pd
import matplotlib.pyplot as plt
merged_df['tpep_pickup_datetime'] = pd.to_datetime(merged_df['tpep_pickup_datetime'])
merged_df['pickup_hour'] = merged_df['tpep_pickup_datetime'].dt.hour
hourly_trip_counts = merged_df['pickup_hour'].value_counts().sort_index()
busiest_hour = hourly_trip_counts.idxmax()
busiest_hour_count = hourly_trip_counts.max()
print(f" Busiest Hour: {busiest_hour}:00 with {busiest_hour_count} trips")
# Plot
plt.figure(figsize=(10,6))
plt.bar(hourly_trip_counts.index, hourly_trip_counts.values, color='orange')
plt.axvline(busiest_hour, color='red', linestyle='--', label=f'Busiest Hour: {busiest_hour}:00')
plt.title('Number of Trips by Hour of the Day')
plt.xlabel('Hour of Day (0–23)')
plt.ylabel('Number of Trips')
plt.xticks(range(0, 24))
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend()
plt.tight_layout()
plt.show()

How does identifying high-traffic, high-demand routes help us?

**3.2.2** <font color = red>[3 marks]</font> <br>
Calculate the number of trips at each hour of the day and visualise them. Find the busiest hour and show the number of trips for that hour.

In [ ]:
# Visualise the number of trips per hour and find the busiest hour
hourly_trip_counts = merged_df['pickup_time_derived'].value_counts().sort_index()
render_bar_plot(hourly_trip_counts.index, hourly_trip_counts.values, 'Number of Trips at Each Hour of the Day', 'Hour of the Day', 'Number of Trips')
busiest_hour = hourly_trip_counts.idxmax()
busiest_hour_trips = hourly_trip_counts.max()
print(f'The busiest hour is {busiest_hour}:00 with {busiest_hour_trips} trips.')

In [ ]:
busiest_hours = hourly_trip_counts.nlargest(5)
print("Actual number of trips in the five busiest hours:")
busiest_hours

Remember, we took a fraction of trips. To find the actual number, you have to scale the number up by the sampling ratio.

**3.2.3** <font color = red>[2 mark]</font> <br>
Find the actual number of trips in the five busiest hours

In [ ]:
# Scale up the number of trips
scaled_trips_per_location = trips_per_location.copy()
scaled_trips_per_location['total_trips'] *= sampling_ratio
sample_fraction =0.1
scaled_trips_per_location.head()

**3.2.4** <font color = red>[3 marks]</font> <br>
Compare hourly traffic pattern on weekdays. Also compare for weekend.

In [ ]:
# Compare traffic trends for the week days and weekends
weekdays_df = merged_df[merged_df['pickup_day_derived'] < 5]  # Monday to Friday
weekends_df = merged_df[merged_df['pickup_day_derived'] >= 5]  # Saturday and Sunday
weekday_hourly_pickups = weekdays_df['pickup_time_derived'].value_counts().sort_index()
weekend_hourly_pickups = weekends_df['pickup_time_derived'].value_counts().sort_index()
plt.figure(figsize=(12, 6))
sns.lineplot(x=weekday_hourly_pickups.index, y=weekday_hourly_pickups.values, marker='o', label='Weekdays')
sns.lineplot(x=weekend_hourly_pickups.index, y=weekend_hourly_pickups.values, marker='o', label='Weekends')
plt.xlabel('Hour of the Day')
plt.ylabel('Number of Trips')
plt.title('Hourly Traffic Pattern: Weekdays vs Weekends')
plt.xticks(range(24))
plt.legend()
plt.show()


What can you infer from the above patterns? How will finding busy and quiet hours for each day help us?

**3.2.5** <font color = red>[3 marks]</font> <br>
Identify top 10 zones with high hourly pickups. Do the same for hourly dropoffs. Show pickup and dropoff trends in these zones.

In [ ]:
# Find top 10 pickup and dropoff zones
pickup_counts = merged_df.groupby(['PULocationID', 'pickup_time_derived']).size().reset_index(name='pickup_count')
top_pickup_zones = pickup_counts.groupby('PULocationID')['pickup_count'].sum().nlargest(10).index
top_pickup_data = pickup_counts[pickup_counts['PULocationID'].isin(top_pickup_zones)]
plt.figure(figsize=(15, 10))
sns.lineplot(data=top_pickup_data, x='pickup_time_derived', y='pickup_count', hue='PULocationID', marker='o')
plt.xlabel('Hour of the Day')
plt.ylabel('Number of Pickups')
plt.title('Hourly Pickup Trends for Top 10 Zones')
plt.legend(title='PULocationID')
plt.xticks(range(24))
plt.show()
dropoff_counts = merged_df.groupby(['DOLocationID', 'pickup_time_derived']).size().reset_index(name='dropoff_count')
top_dropoff_zones = dropoff_counts.groupby('DOLocationID')['dropoff_count'].sum().nlargest(10).index
top_dropoff_data = dropoff_counts[dropoff_counts['DOLocationID'].isin(top_dropoff_zones)]
plt.figure(figsize=(15, 10))
sns.lineplot(data=top_dropoff_data, x='pickup_time_derived', y='dropoff_count', hue='DOLocationID', marker='o')
plt.xlabel('Hour of the Day')
plt.ylabel('Number of Dropoffs')
plt.title('Hourly Dropoff Trends for Top 10 Zones')
plt.legend(title='DOLocationID')
plt.xticks(range(24))
plt.show()

**3.2.6** <font color = red>[3 marks]</font> <br>
Find the ratio of pickups and dropoffs in each zone. Display the 10 highest (pickup/drop) and 10 lowest (pickup/drop) ratios.

In [ ]:
# Find the top 10 and bottom 10 pickup/dropoff ratios
pickup_counts = merged_df['PULocationID'].value_counts().reset_index()
pickup_counts.columns = ['LocationID', 'pickup_count']
dropoff_counts = merged_df['DOLocationID'].value_counts().reset_index()
dropoff_counts.columns = ['LocationID', 'dropoff_count']
pickup_dropoff_counts = pickup_counts.merge(dropoff_counts, on='LocationID', how='outer').fillna(0)
pickup_dropoff_counts['pickup_dropoff_ratio'] = pickup_dropoff_counts['pickup_count'] / pickup_dropoff_counts['dropoff_count']
highest_ratios = pickup_dropoff_counts.nlargest(10, 'pickup_dropoff_ratio')
print("10 Highest Pickup/Dropoff Ratios:")
print(highest_ratios[['LocationID', 'pickup_dropoff_ratio']])
lowest_ratios = pickup_dropoff_counts.nsmallest(10, 'pickup_dropoff_ratio')
print("\n10 Lowest Pickup/Dropoff Ratios:")
print(lowest_ratios[['LocationID', 'pickup_dropoff_ratio']])

**3.2.7** <font color = red>[3 marks]</font> <br>
Identify zones with high pickup and dropoff traffic during night hours (11PM to 5AM)

In [ ]:
# During night hours (11pm to 5am) find the top 10 pickup and dropoff zones
# Note that the top zones should be of night hours and not the overall top zones
night_hours_df = merged_df[(merged_df['pickup_time_derived'] >= 23) | (merged_df['pickup_time_derived'] <= 5)]
night_pickup_counts = night_hours_df['PULocationID'].value_counts().reset_index()
night_pickup_counts.columns = ['LocationID', 'pickup_count']
night_dropoff_counts = night_hours_df['DOLocationID'].value_counts().reset_index()
night_dropoff_counts.columns = ['LocationID', 'dropoff_count']
top_night_pickup_zones = night_pickup_counts.nlargest(10, 'pickup_count')
top_night_dropoff_zones = night_dropoff_counts.nlargest(10, 'dropoff_count')
print("Top 10 Pickup Zones during Night Hours (11 PM to 5 AM):")
print(top_night_pickup_zones)
print("\nTop 10 Dropoff Zones during Night Hours (11 PM to 5 AM):")
print(top_night_dropoff_zones)

Now, let us find the revenue share for the night time hours and the day time hours. After this, we will move to deciding a pricing strategy.

**3.2.8** <font color = red>[2 marks]</font> <br>
Find the revenue share for nighttime and daytime hours.

In [ ]:
# Filter for night hours (11 PM to 5 AM)
night_hours_df = merged_df[(merged_df['pickup_time_derived'] >= 23) | (merged_df['pickup_time_derived'] <= 5)]
night_revenue = night_hours_df['total_amount'].sum()
day_hours_df = merged_df[(merged_df['pickup_time_derived'] >= 6) & (merged_df['pickup_time_derived'] <= 22)]
day_revenue = day_hours_df['total_amount'].sum()
total_revenue = merged_df['total_amount'].sum()
night_revenue_share = (night_revenue / total_revenue) * 100
day_revenue_share = (day_revenue / total_revenue) * 100
labels = ['Nighttime Revenue', 'Daytime Revenue']
sizes = [night_revenue_share, day_revenue_share]
colors = ['#ff9999','#66b3ff']
explode = (0.1, 0)
plt.figure(figsize=(8, 8))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
plt.title('Revenue Share: Nighttime vs Daytime')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

##### Pricing Strategy

**3.2.9** <font color = red>[2 marks]</font> <br>
For the different passenger counts, find the average fare per mile per passenger.

For instance, suppose the average fare per mile for trips with 3 passengers is 3 USD/mile, then the fare per mile per passenger will be 1 USD/mile.

In [ ]:
# Analyse the fare per mile per passenger for different passenger counts
merged_df['fare_per_mile_derived'] = merged_df['fare_amount'] / merged_df['trip_distance']
avg_fare_per_mile = merged_df.groupby('passenger_count')['fare_per_mile_derived'].mean().reset_index()
avg_fare_per_mile['fare_per_mile_per_passenger'] = avg_fare_per_mile['fare_per_mile_derived'] / avg_fare_per_mile['passenger_count']
avg_fare_per_mile

**3.2.10** <font color = red>[3 marks]</font> <br>
Find the average fare per mile by hours of the day and by days of the week

In [ ]:
# Compare the average fare per mile for different days and for different times of the day
avg_fare_per_mile_by_hour = merged_df.groupby('pickup_time_derived')['fare_per_mile_derived'].mean().reset_index()
avg_fare_per_mile_by_day = merged_df.groupby('pickup_day_derived')['fare_per_mile_derived'].mean().reset_index()
day_name_map = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
avg_fare_per_mile_by_day['pickup_day_name'] = avg_fare_per_mile_by_day['pickup_day_derived'].map(day_name_map)
render_line_plot(avg_fare_per_mile_by_hour['pickup_time_derived'], avg_fare_per_mile_by_hour['fare_per_mile_derived'], 'Average Fare per Mile by Hours of the Day', 'Hour of the Day', 'Average Fare per Mile')
render_bar_plot(avg_fare_per_mile_by_day['pickup_day_name'], avg_fare_per_mile_by_day['fare_per_mile_derived'], 'Average Fare per Mile by Days of the Week', 'Day of the Week', 'Average Fare per Mile')

**3.2.11** <font color = red>[3 marks]</font> <br>
Analyse the average fare per mile for the different vendors for different hours of the day

In [ ]:
# Compare fare per mile for different vendors
avg_fare_per_mile_by_vendor_hour = merged_df.groupby(['VendorID', 'pickup_time_derived'])['fare_per_mile_derived'].mean().reset_index()
plt.figure(figsize=(14, 8))
sns.lineplot(x='pickup_time_derived', y='fare_per_mile_derived', hue='VendorID', data=avg_fare_per_mile_by_vendor_hour, marker='o')
plt.xlabel('Hour of the Day')
plt.ylabel('Average Fare per Mile')
plt.title('Average Fare per Mile for Different Vendors by Hour of the Day')
plt.xticks(range(24))
plt.legend(title='VendorID')
plt.show()

**3.2.12** <font color = red>[5 marks]</font> <br>
Compare the fare rates of the different vendors in a tiered fashion. Analyse the average fare per mile for distances upto 2 miles. Analyse the fare per mile for distances from 2 to 5 miles. And then for distances more than 5 miles.


In [ ]:
# Defining distance tiers
distance_tiers = [
    (0, 2),
    (2, 5),
    (5, float('inf'))
]
results = []
for min_distance, max_distance in distance_tiers:
    tier_df = merged_df[(merged_df['trip_distance'] > min_distance) & (merged_df['trip_distance'] <= max_distance)]
    avg_fare_per_mile = tier_df.groupby('VendorID').apply(lambda x: (x['fare_amount'] / x['trip_distance']).mean()).reset_index()
    avg_fare_per_mile.columns = ['VendorID', 'avg_fare_per_mile']
    avg_fare_per_mile['distance_tier'] = f'{min_distance} - {max_distance} miles'
    results.append(avg_fare_per_mile)
fare_rate_comparison = pd.concat(results, ignore_index=True)
fare_rate_comparison

##### Customer Experience and Other Factors

**3.2.13** <font color = red>[5 marks]</font> <br>
Analyse average tip percentages based on trip distances, passenger counts and time of pickup. What factors lead to low tip percentages?

In [ ]:
#  Analyze tip percentages based on distances, passenger counts and pickup times
# Compare trips with tip percentage < 10% to trips with tip percentage > 25%
merged_df['tip_percentage_derived'] = (merged_df['tip_amount'] / merged_df['fare_amount']) * 100
avg_tip_by_distance = merged_df.groupby('trip_distance')['tip_percentage_derived'].mean().reset_index()
avg_tip_by_passenger_count = merged_df.groupby('passenger_count')['tip_percentage_derived'].mean().reset_index()
avg_tip_by_hour = merged_df.groupby('pickup_time_derived')['tip_percentage_derived'].mean().reset_index()
render_line_plot(avg_tip_by_distance['trip_distance'], avg_tip_by_distance['tip_percentage_derived'], 'Average Tip Percentage by Trip Distance', 'Trip Distance (miles)', 'Average Tip Percentage')
render_bar_plot(avg_tip_by_passenger_count['passenger_count'], avg_tip_by_passenger_count['tip_percentage_derived'], 'Average Tip Percentage by Passenger Count', 'Passenger Count', 'Average Tip Percentage')
render_line_plot(avg_tip_by_hour['pickup_time_derived'], avg_tip_by_hour['tip_percentage_derived'], 'Average Tip Percentage by Hour of the Day', 'Hour of the Day', 'Average Tip Percentage')

Additional analysis [optional]: Let's try comparing cases of low tips with cases of high tips to find out if we find a clear aspect that drives up the tipping behaviours

**3.2.14** <font color = red>[3 marks]</font> <br>
Analyse the variation of passenger count across hours and days of the week.

In [ ]:
# See how passenger count varies across hours and days
avg_passenger_count_by_hour = merged_df.groupby('pickup_time_derived')['passenger_count'].mean().reset_index()
avg_passenger_count_by_day = merged_df.groupby('pickup_day_name_derived')['passenger_count'].mean().reset_index()
render_line_plot(avg_passenger_count_by_hour['pickup_time_derived'], avg_passenger_count_by_hour['passenger_count'], 'Average Passenger Count by Hours of the Day', 'Hour of the Day', 'Average Passenger Count')
render_bar_plot(avg_passenger_count_by_day['pickup_day_name_derived'], avg_passenger_count_by_day['passenger_count'], 'Average Passenger Count by Day of the Week', 'Day of the Week', 'Average Passenger Count', show_data_labels=True)

**3.2.15** <font color = red>[2 marks]</font> <br>
Analyse the variation of passenger counts across zones

In [ ]:
# How does passenger count vary across zones
avg_passenger_count_by_zone = merged_df.groupby('PULocationID')['passenger_count'].mean().reset_index()

In [ ]:
# For a more detailed analysis, we can use the zones_with_trips GeoDataFrame
# Create a new column for the average passenger count in each zone.
# Calculate the average passenger count for each zone
avg_passenger_count_by_zone = merged_df.groupby('PULocationID')['passenger_count'].mean().reset_index()

# Merge the average passenger count with the zones_with_trips GeoDataFrame
zones_with_trips = zones_with_trips.merge(avg_passenger_count_by_zone, left_on='LocationID', right_on='PULocationID', how='left')

# Fill NaN values with 0 (for zones with no trips)
zones_with_trips['passenger_count'].fillna(0, inplace=True)

# Plot the average passenger count for each zone using a Geo Plot
zones_with_trips.plot(column='passenger_count', cmap='Blues', legend=True, legend_kwds={'label': "Average Passenger Count", 'orientation': "horizontal"}, figsize=(12, 12))
plt.title('Average Passenger Count by Zone')
plt.show()

Find out how often surcharges/extra charges are applied to understand their prevalance

**3.2.16** <font color = red>[5 marks]</font> <br>
Analyse the pickup/dropoff zones or times when extra charges are applied more frequently

In [ ]:
# How often is each surcharge applied?
surcharge_columns = ['congestion_surcharge', 'improvement_surcharge']
surcharge_frequencies = merged_df[surcharge_columns].apply(lambda x: (x > 0).sum()).reset_index()
surcharge_frequencies.columns = ['Surcharge', 'Frequency']
print("Frequency of each surcharge:")
print(surcharge_frequencies)

In [ ]:
# Analyze the pickup zones where surcharges are applied more frequently
pickup_surcharge_zones = merged_df[merged_df[surcharge_columns].sum(axis=1) > 0].groupby('PULocationID').size().reset_index(name='surcharge_count')
pickup_surcharge_zones = pickup_surcharge_zones.merge(zones[['LocationID', 'zone']], left_on='PULocationID', right_on='LocationID', how='left').sort_values(by='surcharge_count', ascending=False)

# Display the top pickup zones with surcharges
print("Top pickup zones with surcharges:")
print(pickup_surcharge_zones.head(10))

In [ ]:
# Analyze the dropoff zones where surcharges are applied more frequently
dropoff_surcharge_zones = merged_df[merged_df[surcharge_columns].sum(axis=1) > 0].groupby('DOLocationID').size().reset_index(name='surcharge_count')
dropoff_surcharge_zones = dropoff_surcharge_zones.merge(zones[['LocationID', 'zone']], left_on='DOLocationID', right_on='LocationID', how='left').sort_values(by='surcharge_count', ascending=False)

# Display the top dropoff zones with surcharges
print("Top dropoff zones with surcharges:")
print(dropoff_surcharge_zones.head(10))


In [ ]:
# Analyze the times when surcharges are applied more frequently
merged_df['surcharge_applied_derived'] = merged_df[surcharge_columns].sum(axis=1) > 0
surcharge_by_hour = merged_df.groupby('pickup_time_derived')['surcharge_applied_derived'].sum().reset_index()

# Plot the frequency of surcharges by hour of the day
render_bar_plot(surcharge_by_hour['pickup_time_derived'], surcharge_by_hour['surcharge_applied_derived'], 'Frequency of Surcharges by Hour of the Day', 'Hour of the Day', 'Number of Surcharges Applied')

## **4** Conclusion
<font color = red>[15 marks]</font> <br>

### **4.1** Final Insights and Recommendations
<font color = red>[15 marks]</font> <br>

Conclude your analyses here. Include all the outcomes you found based on the analysis.

Based on the insights, frame a concluding story explaining suitable parameters such as location, time of the day, day of the week etc. to be kept in mind while devising a strategy to meet customer demand and optimise supply.

**4.1.1** <font color = red>[5 marks]</font> <br>
Recommendations to optimize routing and dispatching based on demand patterns and operational inefficiencies

Based on the analysis of demand patterns and operational inefficiencies, the company should optimize its fleet management and pricing strategy. During peak hours (6 AM to 10 PM), deploying additional cabs will help reduce passenger wait times and improve fleet utilization. Implementing surge pricing in high-demand areas—such as business districts and transit hubs—will regulate demand while boosting revenue. Additionally, fare adjustments based on time of day and day of the week will help maximize earnings without compromising competitiveness, as evident from the analysis of average fare per mile. Late-night hours (11 PM to 5 AM) require an increased fleet presence in key pickup and drop-off zones to meet peak demand efficiently. Advanced repositioning algorithms can further enhance vehicle distribution during surge periods, ensuring optimal placement to cut down wait times. These strategic measures will drive greater operational efficiency, elevate customer satisfaction, and ultimately improve the company’s profitability

**4.1.2** <font color = red>[5 marks]</font> <br>

Suggestions on strategically positioning cabs across different zones to make best use of insights uncovered by analysing trip trends across time, days and months.

In [ ]:
# Aggregate trip counts and passenger counts by zone and time
demand_data = merged_df.groupby(['PULocationID', 'pickup_time_derived']).agg({
    'passenger_count': 'sum',
    'PULocationID': 'count'
}).rename(columns={'PULocationID': 'trip_count'}).reset_index()

# Pivot the data to create a matrix for the heatmap
heatmap_data = demand_data.pivot(index='PULocationID', columns='pickup_time_derived', values='trip_count')

# Plot the heatmap
plt.figure(figsize=(16, 12))
sns.heatmap(heatmap_data, cmap="crest", linewidths=0.5)
plt.title('Heatmap of Demand Intensity Across Different Zones at Various Times')
plt.xlabel('Hour of the Day')
plt.ylabel('Zone (PULocationID)')
plt.show()

The heatmap analysis reveals distinct taxi demand patterns across different pickup zones and hours of the day. The Y-axis represents various pickup zones (PULocationID), while the X-axis covers the full 24-hour range. Color intensity visually reflects demand levels, with darker blue shades indicating high pickup volumes (typically over 800) and lighter green tones signaling lower activity.

Key insights emerge from the visualization. High-demand zones—such as IDs 132, 138, 142, 163, and 236—consistently show dense taxi activity, especially during morning rush hours (7–10 AM) and evening peaks (4–8 PM). These areas likely represent transportation hubs, commercial districts, or densely populated regions. Conversely, numerous zones display lighter colors, suggesting minimal taxi activity, potentially indicative of residential neighborhoods, less frequented locations, or under-served areas.

Temporal patterns also emerge, revealing demand spikes during traditional commuting hours, reflecting established behavioral trends. While certain zones maintain steady demand throughout the day, most exhibit time-dependent fluctuations, possibly linked to office hours, nightlife activity, or public events.

These insights have practical implications. Taxi operators can use this data to strategically position drivers in high-demand locations at critical times, improving service efficiency. Surge pricing can be applied in busy periods and high-traffic zones to optimize revenue. From an urban planning perspective, these heatmaps highlight mobility hotspots, guiding infrastructure development and transit planning decisions.


**4.1.3** <font color = red>[5 marks]</font> <br>
Propose data-driven adjustments to the pricing strategy to maximize revenue while maintaining competitive rates with other vendors.

The company can refine its pricing strategy using data-driven insights to boost revenue while maintaining competitiveness. A notable seasonal decline occurs between July and September, suggesting the need for promotional fares or targeted discounts to attract more riders and improve fleet utilization during this period. Additionally, the low correlation (0.32) between trip length and fare amount indicates that long trips may be underpriced. Introducing waiting time charges during periods of inactivity—such as traffic stoppages—would ensure fares more accurately reflect trip duration.

Passenger-based pricing offers another optimization opportunity. Current fare structures do not account for the number of riders, meaning a group ride surcharge or discounted bundled rates could better align pricing with resource usage. Furthermore, machine learning models could predict demand elasticity based on location, time, and trip distance, enabling real-time fare adjustments that respond to customer sensitivity without negatively impacting ride volume. Finally, refining distance-based fare calibration using historical ride data would enhance fairness while maximizing revenue for both short and long trips.
